# Bài Tập Thực Hành Buổi 1

## Bài 1: Tính trung bình cộng, trung bình nhân của một dãy số thực

In [58]:
import random

In [59]:
data = []
for i in range(0,1000):
    data.append(random.uniform(1,10))

In [60]:
# Tạo RDD từ danh sách số thực
rddSoThuc = sc.parallelize(data)

# Đếm tổng các số thực
count = rddSoThuc.count()

In [61]:
# Tính trung bình cộng
tong = rddSoThuc.reduce(lambda x, y: x + y)
tbCong = tong / count
print(f"Trung bình cộng của dãy số thực: {tbCong}")

Trung bình cộng của dãy số thực: 5.5946345954187615


In [62]:
# Tính trung bình nhân
import math
log_sum = rddSoThuc.map(lambda x: math.log(x)).reduce(lambda x, y: x + y)
tbNhan = math.exp(log_sum / count)
print(f"Trung bình nhân của dãy số thực: {tbNhan}")

Trung bình nhân của dãy số thực: 4.795619269720676


## Bài 2: Tính phương sai và độ lệch chuẩn của một dãy số thực

In [63]:
# Tính phương sai (variance)
phuongsai = rddSoThuc.map(lambda x: (x - tbCong) ** 2).reduce(lambda x, y: x + y) / count
print(f"Phương sai của dãy số thực: {phuongsai}")

Phương sai của dãy số thực: 7.168650638860481


In [64]:
# Tính độ lệch chuẩn
dolechchuan = phuongsai ** 0.5
print(f"Độ lệch chuẩn: {dolechchuan}")

Độ lệch chuẩn: 2.677433591867496


## Bài 3: Tính tổng của một dãy phân số đọc từ tập tin

#### Cách 1: Thủ công

In [65]:
rddPhanSoData = sc.textFile('data_phanso')
rddPhanSoData.collect()

['2/3', '5/6', '7/8', '1/2', '9/10']

In [66]:
# Hàm tách phân số (tử, mẫu)
def tachPhanSo(chuoi: str):
    items = chuoi.split('/')
    return (int(items[0]), int(items[1]))

In [67]:
# Phân số sau khi parse
rddPhanSo = rddPhanSoData.map(tachPhanSo)
rddPhanSo.collect()

[(2, 3), (5, 6), (7, 8), (1, 2), (9, 10)]

In [68]:
from math import gcd

# Hàm cộng 2 phân số thủ công
def congPhanSo(ps1, ps2):
    tu1, mau1 = ps1
    tu2, mau2 = ps2
    # Quy đồng
    tu = tu1 * mau2 + tu2 * mau1
    mau = mau1 * mau2
    # Rút gọn phân số
    ucln = gcd(tu, mau)
    return (tu // ucln, mau // ucln)

In [69]:
# Tính tổng
tongPhanSo = rddPhanSo.reduce(congPhanSo)

print(f"Tổng của dãy phân số: {tongPhanSo[0]}/{tongPhanSo[1]}")
print(f"Giá trị thập phân: {tongPhanSo[0] / tongPhanSo[1]}")

Tổng của dãy phân số: 151/40
Giá trị thập phân: 3.775


#### Cách 2: sử dụng module fractions để xử lý phân số

In [70]:
from fractions import Fraction

# Đọc dữ liệu từ file
rddPhanSoData = sc.textFile('data_phanso')
print("Dữ liệu gốc:", rddPhanSoData.collect())

# Tách thành phân số bằng Fraction
def tachPhanSo(chuoi: str):
    tu, mau = chuoi.split('/')
    return Fraction(int(tu), int(mau))

rddPhanSo = rddPhanSoData.map(tachPhanSo)
print("Phân số sau khi parse:", rddPhanSo.collect())

# Tính tổng các phân số
tongPhanSo = rddPhanSo.reduce(lambda x, y: x + y)

print(f"Tổng của dãy phân số: {tongPhanSo} (dạng phân số)")
print(f"Giá trị thập phân: {float(tongPhanSo)}")


Dữ liệu gốc: ['2/3', '5/6', '7/8', '1/2', '9/10']
Phân số sau khi parse: [Fraction(2, 3), Fraction(5, 6), Fraction(7, 8), Fraction(1, 2), Fraction(9, 10)]
Tổng của dãy phân số: 151/40 (dạng phân số)
Giá trị thập phân: 3.775
